In [ ]:
import networkx as nx
from gurobipy import Model, GRB

# ================================
# 1. CREAZIONE GRAFO
# ================================
G = nx.DiGraph()
G.add_node("Depot1", type="depot", charging_possible=True)
G.add_node("Depot2", type="depot", charging_possible=True)
G.add_node("Stop1", type="stop", charging_possible=True)
G.add_node("Stop2", type="stop", charging_possible=False)
G.add_node("Stop3", type="stop", charging_possible=True)
G.add_node("Stop4", type="stop", charging_possible=False)
G.add_node("Stop5", type="stop", charging_possible=True)



G.add_edge("Depot1", "Stop1", distance=3)
G.add_edge("Stop1", "Stop2", distance=4)
G.add_edge("Stop2", "Stop3", distance=2)
G.add_edge("Stop3", "Stop5", distance=5)
G.add_edge("Stop5", "Depot2", distance=4)
G.add_edge("Depot1", "Stop4", distance=6)
G.add_edge("Stop4", "Stop5", distance=7)
G.add_edge("Depot2", "Stop3", distance=3)

# ================================
# 2. SET E PARAMETRI
# ================================
R = ["r1"]
B = ["E1"]
C = ["C1"]
T = ["T1"]
N = list(G.nodes)

πr = {"r1": ["Depot", "Stop1", "Stop2", "Depot"]}

# Parametri tecnici
cap = {"E1": 100}
dmax = {"E1": 12}
ctr = {("r1", "Stop1", "E1", "C1"): 5}
ucc = 1  # plug per point

# Costi
cbus = {"E1": 300000}
vcbus = {"E1": 100000}
pc = {"C1": 200}
ccpc = 10000
vcpc = 2000
ccpst = 100000
cltj = 10000
dem = {"r1": 200}

# ================================
# 3. CREAZIONE MODELLO GUROBI
# ================================
m = Model("Mini_EBus")

# Variabili
nbrbc = m.addVar(vtype=GRB.INTEGER, name="nbrbc")        # num bus
yrbc = m.addVar(vtype=GRB.BINARY, name="yrbc")           # usato?
y_s = m.addVar(vtype=GRB.BINARY, name="y_scenario")      # scenario attivo?
npjc = m.addVar(vtype=GRB.INTEGER, name="npjc")          # charging points
ncjc = m.addVar(vtype=GRB.INTEGER, name="ncjc")          # plug devices
βt = m.addVar(vtype=GRB.BINARY, name="βt")               # power station
γtj = m.addVar(vtype=GRB.BINARY, name="γtj")             # collegamento

# ================================
# 4. FUNZIONE OBIETTIVO
# ================================
m.setObjective(
    cap["E1"] * nbrbc
    - 0.01 * cbus["E1"] * nbrbc
    - 0.01 * ccpc * ncjc
    - 0.01 * ccpc * npjc
    - 0.01 * ccpst * βt,
    GRB.MAXIMIZE
)

# ================================
# 5. VINCOLI
# ================================

# Attivazione scenario implica bus
m.addConstr(nbrbc <= 5 * y_s, "BusScenario")

# Se attivo, deve avere plug device
m.addConstr(ncjc >= ucc * y_s, "PlugNeeded")

# Colonnina ha almeno un plug
m.addConstr(npjc >= ncjc, "PointHasPlug")

# Stazione di potenza sufficiente
m.addConstr(pc["C1"] * ncjc <= 800 * βt, "PowerLimit")

# Collegamento solo se stazione usata
m.addConstr(γtj <= βt, "LinkValid")

# Capacità soddisfa domanda (semplificata)
m.addConstr(cap["E1"] * nbrbc >= dem["r1"], "DemandCovered")

# ================================
# 6. RISOLUZIONE
# ================================
m.optimize()

# ================================
# 7. RISULTATI
# ================================
print("\n--- RISULTATI ---")
for v in m.getVars():
    print(f"{v.VarName}: {v.X}")
print(f"Objective Value: {m.ObjVal}")


Set parameter Username
Set parameter LicenseID to value 2643657
Academic license - for non-commercial use only - expires 2026-03-28
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 11th Gen Intel(R) Core(TM) i5-1155G7 @ 2.50GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6 rows, 7 columns and 11 nonzeros
Model fingerprint: 0xda4e1931
Variable types: 0 continuous, 7 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+02]
  Objective range  [1e+02, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+02, 2e+02]
Presolve removed 6 rows and 7 columns
Presolve time: 0.02s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.04 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: -7000 
No other solutions better than -7000

Optimal solution found (toler